<h1>Question 1</h1>

In [103]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [104]:
# Scrap the information from Wikipedia link
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
original_data = BeautifulSoup(source, 'html.parser')
# Create an empty Dataframe with the below column
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [105]:
# Scrap the information from each of the tr and td
table = original_data.find('table')
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        #remove all of the <tr>/<td> and the /n
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [106]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h6>Cleaning of Data</h6>

In [107]:
df=df[df['Borough']!='Not assigned']
df.reset_index(drop = True, inplace = True)
df.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [108]:
# drop the row that have NaN
df = df.dropna()
empty = 'Not assigned'
# drop the row containing information of Not Assigned
df = df[(df['Postalcode'] != empty ) & (df['Borough'] != empty) & (df['Neighborhood'] != empty)]
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<p>Finally, print the number of rows of dataframe</p>

In [110]:
df.shape

(103, 3)

<h1> Question 2 </h1>

Use the csv file to create the dataframe with coordinates:

In [118]:
geospatial_df=pd.read_csv('http://cocl.us/Geospatial_data')
geospatial_df.head()
#Rename the column name for combination of dataframe
geospatial_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
#Combine the two dataframe according to same postalcode 
geo_combined = pd.merge(geospatial_df, df, on='Postalcode')
#Rearrange the sequence of the column
geo_data=geo_combined[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]
geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
